In [13]:


import tkinter as tk
from tkinter import ttk, messagebox
import csv
import os

COLOR_FONDO = "#e6ffe6"  # Verde menta

# ---------- Funciones de datos ----------
def inicializar_csv():
    if not os.path.exists("libros.csv"):
        with open("libros.csv", "w", newline="", encoding="utf-8-sig") as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(["Título", "Autor", "Género", "Año", "Usuario", "Fecha Límite", "Estado"])

def cargar_libros():
    inicializar_csv()
    libros = []
    with open("libros.csv", newline='', encoding='utf-8-sig') as archivo:
        lector = csv.reader(archivo)
        next(lector, None)
        libros = list(lector)
    return libros

def guardar_libro(titulo, autor, genero, anio, usuario, fecha, estado):
    inicializar_csv()
    with open("libros.csv", 'a', newline='', encoding='utf-8-sig') as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow([titulo, autor, genero, anio, usuario, fecha, estado])
    messagebox.showinfo("Éxito", "Libro guardado correctamente.")

# ---------- Ventana de gestión ----------
def ventana_libros():
    ventana = tk.Toplevel()
    ventana.title("Gestión de Libros")
    ventana.geometry("900x500")
    ventana.configure(bg=COLOR_FONDO)

    ttk.Label(ventana, text="Título:", background=COLOR_FONDO).grid(row=0, column=0)
    titulo = ttk.Entry(ventana)
    titulo.grid(row=0, column=1)

    ttk.Label(ventana, text="Autor:", background=COLOR_FONDO).grid(row=1, column=0)
    autor = ttk.Entry(ventana)
    autor.grid(row=1, column=1)

    ttk.Label(ventana, text="Género:", background=COLOR_FONDO).grid(row=2, column=0)
    genero = ttk.Entry(ventana)
    genero.grid(row=2, column=1)

    ttk.Label(ventana, text="Año:", background=COLOR_FONDO).grid(row=3, column=0)
    anio = ttk.Entry(ventana)
    anio.grid(row=3, column=1)

    ttk.Label(ventana, text="Usuario (quien lleva el libro):", background=COLOR_FONDO).grid(row=0, column=2)
    usuario = ttk.Entry(ventana)
    usuario.grid(row=0, column=3)

    ttk.Label(ventana, text="Fecha límite (dd/mm/aaaa):", background=COLOR_FONDO).grid(row=1, column=2)
    fecha = ttk.Entry(ventana)
    fecha.grid(row=1, column=3)

    ttk.Label(ventana, text="Estado:", background=COLOR_FONDO).grid(row=2, column=2)
    estado = ttk.Combobox(ventana, values=["Disponible", "Prestado"], state="readonly")
    estado.grid(row=2, column=3)
    estado.set("Disponible")

    # Tabla de libros
    columnas = ("Título", "Autor", "Género", "Año", "Usuario", "Fecha Límite", "Estado")
    tabla = ttk.Treeview(ventana, columns=columnas, show="headings", height=12)
    for col in columnas:
        tabla.heading(col, text=col)
        tabla.column(col, width=120)
    tabla.grid(row=6, column=0, columnspan=4, padx=10, pady=10)

    # Scrollbar
    scroll = ttk.Scrollbar(ventana, orient="vertical", command=tabla.yview)
    tabla.configure(yscrollcommand=scroll.set)
    scroll.grid(row=6, column=4, sticky="ns")

    def actualizar_tabla():
        tabla.delete(*tabla.get_children())
        for libro in cargar_libros():
            tabla.insert("", tk.END, values=libro)

    def limpiar_campos():
        titulo.delete(0, tk.END)
        autor.delete(0, tk.END)
        genero.delete(0, tk.END)
        anio.delete(0, tk.END)
        usuario.delete(0, tk.END)
        fecha.delete(0, tk.END)
        estado.set("Disponible")

    def validar_campos():
        if not all([titulo.get().strip(), autor.get().strip(), genero.get().strip(), anio.get().strip()]):
            messagebox.showwarning("Campos vacíos", "Título, autor, género y año son obligatorios.")
            return False
        if not anio.get().isdigit():
            messagebox.showwarning("Año inválido", "El año debe ser numérico.")
            return False
        return True

    def guardar():
        if not validar_campos():
            return
        guardar_libro(
            titulo.get(), autor.get(), genero.get(), anio.get(),
            usuario.get(), fecha.get(), estado.get()
        )
        limpiar_campos()
        actualizar_tabla()

    def seleccionar_libro(event):
        seleccionado = tabla.focus()
        if seleccionado:
            valores = tabla.item(seleccionado, 'values')
            titulo.delete(0, tk.END); autor.delete(0, tk.END); genero.delete(0, tk.END)
            anio.delete(0, tk.END); usuario.delete(0, tk.END); fecha.delete(0, tk.END)
            titulo.insert(0, valores[0]); autor.insert(0, valores[1]); genero.insert(0, valores[2])
            anio.insert(0, valores[3]); usuario.insert(0, valores[4]); fecha.insert(0, valores[5])
            estado.set(valores[6])

    def editar():
        seleccionado = tabla.focus()
        if not seleccionado:
            messagebox.showwarning("Atención", "Selecciona un libro para editar.")
            return
        if not validar_campos():
            return
        valores = tabla.item(seleccionado, 'values')
        libros = cargar_libros()
        nuevo = [titulo.get(), autor.get(), genero.get(), anio.get(),
                 usuario.get(), fecha.get(), estado.get()]
        libros = [nuevo if libro == list(valores) else libro for libro in libros]
        with open("libros.csv", 'w', newline='', encoding='utf-8-sig') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(["Título", "Autor", "Género", "Año", "Usuario", "Fecha Límite", "Estado"])
            escritor.writerows(libros)
        messagebox.showinfo("Éxito", "Libro editado correctamente.")
        limpiar_campos()
        actualizar_tabla()

    def eliminar():
        seleccionado = tabla.focus()
        if not seleccionado:
            messagebox.showwarning("Atención", "Selecciona un libro para eliminar.")
            return
        if not messagebox.askyesno("Confirmar", "¿Seguro que deseas eliminar este libro?"):
            return
        valores = tabla.item(seleccionado, 'values')
        libros = cargar_libros()
        libros = [libro for libro in libros if libro != list(valores)]
        with open("libros.csv", 'w', newline='', encoding='utf-8-sig') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerow(["Título", "Autor", "Género", "Año", "Usuario", "Fecha Límite", "Estado"])
            escritor.writerows(libros)
        messagebox.showinfo("Éxito", "Libro eliminado correctamente.")
        limpiar_campos()
        actualizar_tabla()

    ttk.Button(ventana, text="Guardar", command=guardar).grid(row=4, column=0, pady=10)
    ttk.Button(ventana, text="Editar", command=editar).grid(row=4, column=1)
    ttk.Button(ventana, text="Eliminar", command=eliminar).grid(row=4, column=2)
    ttk.Button(ventana, text="Refrescar", command=actualizar_tabla).grid(row=4, column=3)

    tabla.bind("<ButtonRelease-1>", seleccionar_libro)
    actualizar_tabla()

# ---------- Menú principal ----------
def menu_principal():
    raiz = tk.Tk()
    raiz.title("Gestor de Biblioteca")
    raiz.geometry("600x400")
    raiz.configure(bg=COLOR_FONDO)

    marco = tk.Frame(raiz, bg=COLOR_FONDO)
    marco.place(relx=0.5, rely=0.5, anchor="center")

    # 🟢 Mensaje de bienvenida restaurado
    ttk.Label(
        marco,
        text="📚 Bienvenida al Gestor de Biblioteca 📚",
        font=("Arial", 16, "bold"),
        background=COLOR_FONDO,
        foreground="#006600"
    ).pack(pady=20)

    ttk.Button(marco, text="Gestionar Libros", command=ventana_libros).pack(pady=10)
    ttk.Button(marco, text="Salir", command=lambda: salir_aplicacion(raiz)).pack(pady=10)

    raiz.mainloop()

def salir_aplicacion(ventana):
    if messagebox.askyesno("Salir", "¿Seguro que deseas cerrar el gestor?"):
        ventana.destroy()

menu_principal()